In [1]:
import requests
import string
from bs4 import BeautifulSoup
import time
from pprint import pprint
import json
import random
from bs4 import BeautifulSoup, Comment
import re

# Part three will use the player_dict.json file to crawl through the players and gather all college data for players who were drafted into the NFL.

In [2]:
'''
Load in player_dict.json file only after part 1 and part 2 have completed.
'''

with open('player_dict.json') as json_file:
    player_dict = json.load(json_file)

In [10]:
def get_position(soup) -> str:
    '''
    Function that takes input of a beautiful soup object of a single player.  
    Returns the player's position as a string.
    '''
    for row in soup.find_all("p"):
        if row.text.startswith("School:"):
            temp = row.text.split()
            while len(temp) > 0:
                if temp[0] == "Position:":
                    temp.pop(0)
                    return temp[0]
                else:
                    temp.pop(0)
            return None

In [11]:
#function to get draft information such as what round were they taken and by which team
def get_drafted(soup):
    '''
    Function that takes input of a beautiful soup object of a single player.
    returns a dictionary object of their draft information.
    Dependency on function get_position
    '''
    temp_dict = {}
    for row in soup.find_all("p"):
        if  row.text.startswith("Draft:") and "School" not in row.text:
            temp_dict["round"] = row.text.split(" round,")[0].split("Draft: ")[1][:-2]
            temp_dict["overallPick"] = row.text.split(", ")[1].split(" overall")[0][:-2]
            temp_dict["draftedBy"] = row.text.split("by the ")[1]
            temp_dict["draftedYear"] = row.text.split("of the ")[1].split(" ")[0]
    temp_dict["Position"] = get_position(soup)       
    return temp_dict

In [12]:
def get_tbody(table):
    '''
    Function that takes an input of a beautiful soup object and extracts the data table body information.
    Stores and returns the information of the body table into a dictionary.
    '''
    
    temp_dict = {}
    tbody = table.find('tbody')
    tr_body = tbody.find_all('tr')
    
    for trb in tr_body:
        th = trb.find('th')
        temp_dict[th.get_text()] = {}

        for td in trb.find_all('td'):
            temp_dict[th.get_text()][td.get('data-stat')] = td.get_text()
            
    return temp_dict

In [13]:
def get_tfoot(table):
    '''
    Function that takes an input of a beautiful soup object and extracts the data table footer information.
    Stores and returns the information of the body table into a dictionary.
    '''
    
    temp_dict = {}
    tfoot = table.find('tfoot')
    thf = tfoot.find('th')

    for tdf in tfoot.find_all('td'):
        temp_dict[tdf.get('data-stat')] = tdf.get_text()
    return temp_dict

In [14]:
#Below is for testing purposes only and can be ignored. 

'''
test_dict = {"mahomes": {
    "college_link": 'https://www.sports-reference.com/cfb/players/patrick-mahomes-1.html',
    "pro_link" : "https://www.pro-football-reference.com/players/M/MahoPa00.htm?" \
    "__hstc=213859787.d9e6ba03be7c2728b9a1f145105b0a87.1633525196065.1634934599787.1635297119111.19&__hssc=213859787.2.1635297119111&__hsfp=2049394679"
    
}}

request = requests.get(test_dict["mahomes"]["college_link"])
soup = BeautifulSoup(request.text, "html.parser")
test_dict["mahomes"]["draftData"] = get_drafted(soup)
tables = soup.find_all(class_ = "table_wrapper")
    
for i in range(len(tables)):
    table = tables[i]
    key = table.get('id')
    thead = table.find("thead")
    print(type(thead),"----------------\n")
    if thead == None:
        comment = table.find(text=lambda text: isinstance(text, Comment))
        table = BeautifulSoup(comment , 'html.parser')
        thead = table.find("thead")
    print("Table key is", key)    
    test_dict["mahomes"][key] = get_tbody(table)
    test_dict["mahomes"][key]['career'] = get_tfoot(table)
    
#pprint(test_dict)
'''

'\ntest_dict = {"mahomes": {\n    "college_link": \'https://www.sports-reference.com/cfb/players/patrick-mahomes-1.html\',\n    "pro_link" : "https://www.pro-football-reference.com/players/M/MahoPa00.htm?"     "__hstc=213859787.d9e6ba03be7c2728b9a1f145105b0a87.1633525196065.1634934599787.1635297119111.19&__hssc=213859787.2.1635297119111&__hsfp=2049394679"\n    \n}}\n\nrequest = requests.get(test_dict["mahomes"]["college_link"])\nsoup = BeautifulSoup(request.text, "html.parser")\ntest_dict["mahomes"]["draftData"] = get_drafted(soup)\ntables = soup.find_all(class_ = "table_wrapper")\n    \nfor i in range(len(tables)):\n    table = tables[i]\n    key = table.get(\'id\')\n    thead = table.find("thead")\n    print(type(thead),"----------------\n")\n    if thead == None:\n        comment = table.find(text=lambda text: isinstance(text, Comment))\n        table = BeautifulSoup(comment , \'html.parser\')\n        thead = table.find("thead")\n    print("Table key is", key)    \n    test_dict["m

In [15]:
def get_data(collegeUrl:str):
    '''
    Takes an input of a college player URL and returns a dictionary of their drafted information, and all
    data tables on the page.
    Dependency on functions get_position, get_drafted, get_tbody, get_tfoot
    '''
    
    college_dict = {}
    request = requests.get(collegeUrl)
    soup = BeautifulSoup(request.text, "html.parser")
    college_dict["draftData"] = get_drafted(soup)
    tables = soup.find_all(class_ = "table_wrapper")

    for i in range(len(tables)):
        table = tables[i]
        key = table.get('id')
        thead = table.find("thead")
        if thead == None:
            comment = table.find(text=lambda text: isinstance(text, Comment))
            table = BeautifulSoup(comment , 'html.parser')
            thead = table.find("thead")  
        college_dict[key] = get_tbody(table)
        college_dict[key]['career'] = get_tfoot(table)

    return college_dict

In [17]:
'''
test run on an example URL of the above function: get_data
'''

url = "https://www.sports-reference.com/cfb/players/jamaal-charles-1.html"
test = get_data(url)
pprint(test)

{'all_defense': {'*2005': {'class': 'FR',
                           'conf_abbr': 'Big 12',
                           'def_int': '',
                           'def_int_td': '',
                           'def_int_yds': '',
                           'def_int_yds_per_int': '',
                           'fumbles_forced': '',
                           'fumbles_rec': '',
                           'fumbles_rec_td': '',
                           'fumbles_rec_yds': '',
                           'g': '13',
                           'pass_defended': '',
                           'pos': 'RB',
                           'sacks': '0.0',
                           'school_name': 'Texas',
                           'tackles_assists': '0',
                           'tackles_loss': '0.0',
                           'tackles_solo': '1',
                           'tackles_total': '1'},
                 '*2006': {'class': 'SO',
                           'conf_abbr': 'Big 12',
                

In [19]:
'''
Main code that will crawl all players in the player_dict object and gather all of the table and draft data
for players that were drafted.
Dependency on functions get_data, get_position, get_drafted, get_tbody, get_tfoot
'''

for letter in player_dict:
    print("Starting at letter", letter)
    
    for i in range(len(player_dict[letter]['playerIds'])):
        if 'collegeData' not in player_dict[letter]['Body'][i].keys():
            url = player_dict[letter]['Body'][i]['collegeLink']
            request = requests.get(url)
            soup = BeautifulSoup(request.text, "html.parser")
            if 'draftedBy' not in get_drafted(soup):
                continue
            else:    
                print("Player:", player_dict[letter]['playerIds'][i])
                player_dict[letter]['Body'][i]["collegeData"] = get_data(url)

            time.sleep(.3)


Starting at letter a
Player: rodney-achter-1
Player: kenneth-acker-1
Player: mike-adamle-1
Player: blue-adams-1
Player: bruce-adams-1
Player: curtis-adams-1
Player: davante-adams-1
Player: david-adams-1
Player: dick-adams-1
Player: gaines-adams-1
Player: george-adams-3
Player: jamal-adams-1
Player: jerell-adams-1
Player: joe-adams-1
Player: john-adams-5
Player: keion-adams-1
Player: keith-adams-1
Player: matthew-adams-1
Player: mike-adams-1
Player: montravius-adams-1
Player: rodney-adams-1
Player: sam-adams-1
Player: stefon-adams-1
Player: titus-adams-1
Player: willis-adams-1
Player: joseph-addai-1


KeyboardInterrupt: 

In [25]:
'''
Save the player_dict object unique json object.  This new file will be used for transformation into Normalized tables.
'''

with open("player_dict-college-withCareerwithYear.json", 'w') as outfile:
        json.dump(player_dict, outfile)

In [13]:
'''
Check counts of the player_dict object to validate results.
'''


total = 0

for letter in player_dict:
    print("Starting at letter", letter)
    letterCount = 0
    for i in range(len(player_dict[letter]['playerIds'])):
        
        if 'collegeData' in player_dict[letter]['Body'][i].keys():
            letterCount += 1
    print(f"Valid data counts for letter {letter}: {letterCount}")
    total+=letterCount
    print(f"New total data counts: {total}\n---------------------------------------\n")

print("Grand Total count of valid college data:", total)

Starting at letter a
Valid data counts for letter a: 440
New total data counts: 440
---------------------------------------

Starting at letter b
Valid data counts for letter b: 1372
New total data counts: 1812
---------------------------------------

Starting at letter c
Valid data counts for letter c: 959
New total data counts: 2771
---------------------------------------

Starting at letter d
Valid data counts for letter d: 623
New total data counts: 3394
---------------------------------------

Starting at letter e
Valid data counts for letter e: 245
New total data counts: 3639
---------------------------------------

Starting at letter f
Valid data counts for letter f: 445
New total data counts: 4084
---------------------------------------

Starting at letter g
Valid data counts for letter g: 658
New total data counts: 4742
---------------------------------------

Starting at letter h
Valid data counts for letter h: 1048
New total data counts: 5790
--------------------------------